<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/obj_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Object Detection Model Training Notebook

structure of project:

```
training-demo/
├─ annotations/
│  └─ label_map.pbtxt
├─ exported-models/
├─ images/
│  ├─ test/
│  └─ train/
├─ models/
│  └─ ${MODEL}/
│     └─ pipeline.config
└─ pre-trained-models/
   └─ ${MODEL}/
```

where `${MODEL}` is a model from the [object detection zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

## TensorFlow Object Detection API Installation

mount data from google drive

In [1]:
workdir = '/content/workspace/MyDrive/tensorflow/power-port-targeting/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

Mounted at /content/workspace


clone tf models repo & utility repo (if not already present)

In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

Cloning into 'models'...
remote: Enumerating objects: 3344, done.
remote: Counting objects: 100% (3344/3344), done.
remote: Compressing objects: 100% (2780/2780), done.
remote: Total 3344 (delta 882), reused 1391 (delta 513), pack-reused 0
Receiving objects: 100% (3344/3344), 34.34 MiB | 26.18 MiB/s, done.
Resolving deltas: 100% (882/882), done.
Cloning into '/content/utils'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 0), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


install object_detection package

In [3]:
%%bash

# update pip
pip install --upgrade pip

# check opencv dependency versions
pip uninstall opencv-python-headless==4.5.5.62 
pip install opencv-python-headless==4.1.2.30

# compile protos and install object_detection
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install . --quiet

  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.6/237.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


test installation

In [4]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-05-16 11:30:34.917252: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0516 11:30:35.245603 140715760068480 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.92s
I0516 11:30:35.695366 140715760068480 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 3.92s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.62s
I0516 11:30:36.314440 140715760068480 test

check tensorflow version

In [5]:
import tensorflow as tf
print(tf.__version__)

2.8.0


Data Pre-Processing

create tfrecords

In [ ]:
from time import time
os.chdir(workdir)
start = time()

# Generate TFRecord for Training Data
!python /content/utils/scripts/preprocessing/generate-tfrecord.py \
    -x images/train \
    -l annotations/label_map.pbtxt \
    -o annotations/train.record

# Generate TFRecord for Validation Data
!python /content/utils/scripts/preprocessing/generate-tfrecord.py \
    -x images/test \
    -l annotations/label_map.pbtxt \
    -o annotations/test.record

print('records generated in {}s'.format(time()-start))

## Training

start tensroboard monitoring

In [ ]:
os.chdir(workdir)
%load_ext tensorboard
%tensorboard --logdir=models/{mname}

start training job

In [ ]:
os.chdir(workdir)
!python /content/models/research/object_detection/model_main_tf2.py \
    --include masks \
    --model_dir=models/{mname} \
    --pipeline_config_path=models/{mname}/pipeline.config

## Export Trained Model

export model

In [ ]:
os.chdir(workdir)
!python /tf/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path ./models/{model})/pipeline.config \
    --trained_checkpoint_dir ./models/{model}/ \
    --output_directory ./exported-models/trained_model